# [Heritiana Daniel Andriasolofo](handriasolofo@aimsammi.org)


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [1]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [2]:
!rm data.zip train* valid*
!wget -O data.zip https://github.com/heritiana-aimsammi-sn2022/NLP_week1/blob/main/Lab3/data/data.zip?raw=true
!unzip data.zip
!rm data.zip

rm: cannot remove 'data.zip': No such file or directory
--2022-05-16 23:58:51--  https://github.com/heritiana-aimsammi-sn2022/NLP_week1/blob/main/Lab3/data/data.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/heritiana-aimsammi-sn2022/NLP_week1/raw/main/Lab3/data/data.zip [following]
--2022-05-16 23:58:52--  https://github.com/heritiana-aimsammi-sn2022/NLP_week1/raw/main/Lab3/data/data.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/heritiana-aimsammi-sn2022/NLP_week1/main/Lab3/data/data.zip [following]
--2022-05-16 23:58:53--  https://raw.githubusercontent.com/heritiana-aimsammi-sn2022/NLP_week1/main/Lab3/data/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.11

In [3]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [4]:
print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [5]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    for sentence in data:
        sentence_with_unk = []
        for word in sentence:
            new_word = word if mincount < vocab[word] else "<unk>"
            sentence_with_unk.append(new_word)
        data_with_unk.append(sentence_with_unk)
    
    return data_with_unk

In [6]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
print(train_data[0])

remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [7]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for k in range(1,n+1):
            # build counts for k-gram where k start from 1 to n.
            for i in range(len(sentence)- k + 1):
                k_words = sentence[i:i+k]
                context = k_words[:-1]
                word = k_words[-1]
                counts[context][word] += 1
        
    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!
    for context in counts.keys():
        ## FILL CODE
        n_context = sum(counts[context].values()) # the number of time that context appeared 
        for word in counts[context].keys():
            prob[context][word] = counts[context][word]/n_context

    return prob

In [8]:
# RUN TO BUILD NGRAM MODEL
n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  2


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [9]:
def get_prob(model, context, w):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    
    context = tuple(context)    
    context_size = len(context)
    
    prob = model[context][w]
    if prob == 0 and context_size>0 :
        prob = 0.4 * get_prob(model, context[1:], w)
    
    return prob


In [10]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    n_word = 0.0
    log_perp = 0.0
    for sentence in data:
        sentence = tuple(sentence)
        len_sntnc = len(sentence)
        for i in range(len_sntnc):
            start = max(0, i - n + 1)
            context = sentence[start:i]
            word = sentence[i]
            n_word += 1
            prob = get_prob(model, context, word)

            log_perp += np.log(prob)

    perp = np.exp(-log_perp/n_word)
        
    return perp

In [11]:
# COMPUTE PERPLEXITY ON VALIDATION SET
print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 47.5824831044242


In [12]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    context = tuple(context)
    words_and_probs = model[context]
    
    return words_and_probs

In [13]:
def generate(model, max_len = 100):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<max_len:
        ## FILL CODE
        available_choice = [0,1,2,3,4,5]
        while True:
            context_size = np.random.choice(available_choice)
            context = tuple(sentence)[-context_size:]
            sample = model[context]
            words = list(sample.keys())
            if len(words) != 0:
                break
            available_choice.remove(context_size)
        prob = list(sample.values())
        word = np.random.choice(words, p = prob)#.item(0)
        sentence.append(word)
        
                
    return " ".join(sentence)

In [14]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: \n\t",generate(model))

Generated sentence: 
	 <s> tom and mary fell in a <unk> <unk> </s>


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [15]:
train_data2, vocab2 = load_data("train2.txt")
train_data2 = remove_rare_words(train_data2, vocab2, mincount = 1)
n = 2
model2 = build_ngram(train_data2, n)

In [16]:
for i in range(5):
    print(f"\n\nTEXT {i} :\t",generate(model, 20))



TEXT 0 :	 <s> how long is <unk> </s>


TEXT 1 :	 <s> if you said that tom filled with him. </s>


TEXT 2 :	 <s> tom was stealing from running in <unk> </s>


TEXT 3 :	 <s> tell tom sat in hand, if they want to the chance that'll happen. </s>


TEXT 4 :	 <s> road of tree he went fishing with the <unk> as big picture. </s>
